In [1]:
import os
import sys
import collections
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import pandas as pd
import numpy as np
import time

In [2]:
os.listdir("./bert-master")

['extract_features.py',
 'optimization_test.py',
 '__init__.py',
 'run_pretraining.py',
 'optimization.py',
 'create_pretraining_data.py',
 'run_classifier.py',
 '.gitignore',
 '__pycache__',
 'modeling.py',
 'tokenization.py',
 'tokenization_test.py',
 'sample_text.txt',
 'run_squad.py',
 'multilingual.md',
 'run_classifier_with_tfhub.py',
 'predicting_movie_reviews_with_bert_on_tf_hub.ipynb',
 'LICENSE',
 'requirements.txt',
 'modeling_test.py',
 'CONTRIBUTING.md',
 'README.md']

In [3]:
# BERT files

sys.path.insert(0, './bert-master')

from run_classifier import *
import modeling
import optimization
import tokenization

In [4]:
# import data

train=pd.read_csv('./data/train.csv')
test=pd.read_csv('./data/test.csv')

# remove new lines etc.

train['comment_text'] = train['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
test['comment_text'] = test['comment_text'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)

# force train into cola format, test is fine as it is

train['dummy_1'] = 'meh'
train['dummy_2'] = '*'

train = train[['dummy_1','target','dummy_2','comment_text']]
train['target'] = np.where(train['target']>=0.5,1,0)

#train = train.sample(frac=0.33)

# export as tab seperated

train.to_csv('train.tsv', sep='\t', index=False, header=False)
test.to_csv('test.tsv', sep='\t', index=False, header=True)

In [5]:
task_name = 'cola'
bert_config_file = './uncased_L-12_H-768_A-12/bert_config.json'
vocab_file = './uncased_L-12_H-768_A-12/vocab.txt'
init_checkpoint = './uncased_L-12_H-768_A-12/bert_model.ckpt'
data_dir = './'
output_dir = './'
do_lower_case = True
max_seq_length = 72
do_train = True
do_eval = False
do_predict = False
train_batch_size = 32
eval_batch_size = 32
predict_batch_size = 32
learning_rate = 2e-5 
num_train_epochs = 1.0
warmup_proportion = 0.1
use_tpu = False
master = None
save_checkpoints_steps = 99999999 # <----- don't want to save any checkpoints
iterations_per_loop = 1000
num_tpu_cores = 8
tpu_cluster_resolver = None

In [6]:
start = time.time()
print("--------------------------------------------------------")
print("Starting training ...")
print("--------------------------------------------------------")

--------------------------------------------------------
Starting training ...
--------------------------------------------------------


In [7]:
bert_config = modeling.BertConfig.from_json_file(bert_config_file)

processor = ColaProcessor()
label_list = processor.get_labels()

tokenizer = tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tpu_cluster_resolver = None
is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2

run_config = tf.contrib.tpu.RunConfig(
  cluster=tpu_cluster_resolver,
  master=master,
  model_dir=output_dir,
  save_checkpoints_steps=save_checkpoints_steps,
  tpu_config=tf.contrib.tpu.TPUConfig(
      iterations_per_loop=iterations_per_loop,
      num_shards=num_tpu_cores,
      per_host_input_for_training=is_per_host))

train_examples = processor.get_train_examples(data_dir)
num_train_steps = int(len(train_examples) / train_batch_size * num_train_epochs)
num_warmup_steps = int(num_train_steps * warmup_proportion)

model_fn = model_fn_builder(
      bert_config=bert_config,
      num_labels=len(label_list),
      init_checkpoint=init_checkpoint,
      learning_rate=learning_rate,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps,
      use_tpu=use_tpu,
      use_one_hot_embeddings=use_tpu)

estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=use_tpu,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=train_batch_size)
      
      
train_file = os.path.join(output_dir, "train.tf_record")

file_based_convert_examples_to_features(
    train_examples, label_list, max_seq_length, tokenizer, train_file)

tf.logging.info("***** Running training *****")
tf.logging.info("  Num examples = %d", len(train_examples))
tf.logging.info("  Batch size = %d", train_batch_size)
tf.logging.info("  Num steps = %d", num_train_steps)

train_input_fn = file_based_input_fn_builder(
    input_file=train_file,
    seq_length=max_seq_length,
    is_training=True,
    drop_remainder=True)
    
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': './', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 99999999, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1b058564a8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '

INFO:tensorflow:Writing example 510000 of 1804874
INFO:tensorflow:Writing example 520000 of 1804874
INFO:tensorflow:Writing example 530000 of 1804874
INFO:tensorflow:Writing example 540000 of 1804874
INFO:tensorflow:Writing example 550000 of 1804874
INFO:tensorflow:Writing example 560000 of 1804874
INFO:tensorflow:Writing example 570000 of 1804874
INFO:tensorflow:Writing example 580000 of 1804874
INFO:tensorflow:Writing example 590000 of 1804874
INFO:tensorflow:Writing example 600000 of 1804874
INFO:tensorflow:Writing example 610000 of 1804874
INFO:tensorflow:Writing example 620000 of 1804874
INFO:tensorflow:Writing example 630000 of 1804874
INFO:tensorflow:Writing example 640000 of 1804874
INFO:tensorflow:Writing example 650000 of 1804874
INFO:tensorflow:Writing example 660000 of 1804874
INFO:tensorflow:Writing example 670000 of 1804874
INFO:tensorflow:Writing example 680000 of 1804874
INFO:tensorflow:Writing example 690000 of 1804874
INFO:tensorflow:Writing example 700000 of 1804874


INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (30522, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = ber

INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_4/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflo

INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_9/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKP

INFO:tensorflow:global_step/sec: 2.94901
INFO:tensorflow:examples/sec: 94.3683
INFO:tensorflow:global_step/sec: 2.93577
INFO:tensorflow:examples/sec: 93.9447
INFO:tensorflow:global_step/sec: 2.88391
INFO:tensorflow:examples/sec: 92.285
INFO:tensorflow:global_step/sec: 2.88456
INFO:tensorflow:examples/sec: 92.306
INFO:tensorflow:global_step/sec: 2.88297
INFO:tensorflow:examples/sec: 92.255
INFO:tensorflow:global_step/sec: 2.90215
INFO:tensorflow:examples/sec: 92.8688
INFO:tensorflow:global_step/sec: 2.90369
INFO:tensorflow:examples/sec: 92.9179
INFO:tensorflow:global_step/sec: 2.94316
INFO:tensorflow:examples/sec: 94.181
INFO:tensorflow:global_step/sec: 2.94251
INFO:tensorflow:examples/sec: 94.1603
INFO:tensorflow:global_step/sec: 2.9483
INFO:tensorflow:examples/sec: 94.3455
INFO:tensorflow:global_step/sec: 2.93635
INFO:tensorflow:examples/sec: 93.9632
INFO:tensorflow:global_step/sec: 2.92543
INFO:tensorflow:examples/sec: 93.6139
INFO:tensorflow:global_step/sec: 2.91978
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 2.87409
INFO:tensorflow:examples/sec: 91.971
INFO:tensorflow:global_step/sec: 2.88658
INFO:tensorflow:examples/sec: 92.3707
INFO:tensorflow:global_step/sec: 2.88935
INFO:tensorflow:examples/sec: 92.4592
INFO:tensorflow:global_step/sec: 2.89721
INFO:tensorflow:examples/sec: 92.7106
INFO:tensorflow:global_step/sec: 2.87978
INFO:tensorflow:examples/sec: 92.1531
INFO:tensorflow:global_step/sec: 2.88937
INFO:tensorflow:examples/sec: 92.4598
INFO:tensorflow:global_step/sec: 2.89271
INFO:tensorflow:examples/sec: 92.5666
INFO:tensorflow:global_step/sec: 2.88427
INFO:tensorflow:examples/sec: 92.2965
INFO:tensorflow:global_step/sec: 2.86533
INFO:tensorflow:examples/sec: 91.6906
INFO:tensorflow:global_step/sec: 2.86828
INFO:tensorflow:examples/sec: 91.785
INFO:tensorflow:global_step/sec: 2.87656
INFO:tensorflow:examples/sec: 92.05
INFO:tensorflow:global_step/sec: 2.8848
INFO:tensorflow:examples/sec: 92.3135
INFO:tensorflow:global_step/sec: 2.86233
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 2.93072
INFO:tensorflow:examples/sec: 93.7831
INFO:tensorflow:global_step/sec: 2.89666
INFO:tensorflow:examples/sec: 92.6931
INFO:tensorflow:global_step/sec: 2.9063
INFO:tensorflow:examples/sec: 93.0016
INFO:tensorflow:global_step/sec: 2.91431
INFO:tensorflow:examples/sec: 93.258
INFO:tensorflow:global_step/sec: 2.91013
INFO:tensorflow:examples/sec: 93.124
INFO:tensorflow:global_step/sec: 2.91125
INFO:tensorflow:examples/sec: 93.1601
INFO:tensorflow:global_step/sec: 2.9227
INFO:tensorflow:examples/sec: 93.5263
INFO:tensorflow:global_step/sec: 2.91471
INFO:tensorflow:examples/sec: 93.2708
INFO:tensorflow:global_step/sec: 2.90565
INFO:tensorflow:examples/sec: 92.9808
INFO:tensorflow:global_step/sec: 2.9281
INFO:tensorflow:examples/sec: 93.6991
INFO:tensorflow:global_step/sec: 2.91292
INFO:tensorflow:examples/sec: 93.2135
INFO:tensorflow:global_step/sec: 2.89158
INFO:tensorflow:examples/sec: 92.5305
INFO:tensorflow:global_step/sec: 2.90636
INFO:tensorflow:

INFO:tensorflow:global_step/sec: 2.91502
INFO:tensorflow:examples/sec: 93.2806
INFO:tensorflow:global_step/sec: 2.9212
INFO:tensorflow:examples/sec: 93.4785
INFO:tensorflow:global_step/sec: 2.9163
INFO:tensorflow:examples/sec: 93.3216
INFO:tensorflow:global_step/sec: 2.91849
INFO:tensorflow:examples/sec: 93.3915
INFO:tensorflow:global_step/sec: 2.9355
INFO:tensorflow:examples/sec: 93.9359
INFO:tensorflow:global_step/sec: 2.91848
INFO:tensorflow:examples/sec: 93.3914
INFO:tensorflow:global_step/sec: 2.93211
INFO:tensorflow:examples/sec: 93.8276
INFO:tensorflow:global_step/sec: 2.91129
INFO:tensorflow:examples/sec: 93.1612
INFO:tensorflow:global_step/sec: 2.9372
INFO:tensorflow:examples/sec: 93.9905
INFO:tensorflow:global_step/sec: 2.92786
INFO:tensorflow:examples/sec: 93.6917
INFO:tensorflow:global_step/sec: 2.90215
INFO:tensorflow:examples/sec: 92.8686
INFO:tensorflow:global_step/sec: 2.92752
INFO:tensorflow:examples/sec: 93.6806
INFO:tensorflow:global_step/sec: 2.92959
INFO:tensorflow

In [8]:
end = time.time()
print("--------------------------------------------------------")
print("Training complete in ", end - start, " seconds")
print("--------------------------------------------------------")

--------------------------------------------------------
Training complete in  14032.059273958206  seconds
--------------------------------------------------------


In [9]:
def file_based_input_fn_builder(input_file, seq_length, is_training,
                                drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  name_to_features = {
      "input_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "input_mask": tf.FixedLenFeature([seq_length], tf.int64),
      "segment_ids": tf.FixedLenFeature([seq_length], tf.int64),
      "label_ids": tf.FixedLenFeature([], tf.int64),
      "is_real_example": tf.FixedLenFeature([], tf.int64),
  }

  def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)

    # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
    # So cast all int64 to int32.
    for name in list(example.keys()):
      t = example[name]
      if t.dtype == tf.int64:
        t = tf.to_int32(t)
      example[name] = t

    return example

  def input_fn(params):
    """The actual input function."""
    
    #batch_size = params["batch_size"]
    batch_size = 64 # <----- hardcoded batch_size added here 
    
    # For training, we want a lot of parallel reading and shuffling.
    # For eval, we want no shuffling and parallel reading doesn't matter.
    d = tf.data.TFRecordDataset(input_file)
    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.apply(
        tf.contrib.data.map_and_batch(
            lambda record: _decode_record(record, name_to_features),
            batch_size=batch_size,
            drop_remainder=drop_remainder))

    return d

  return input_fn

In [10]:
start = time.time()
print("--------------------------------------------------------")
print("Starting inference ...")
print("--------------------------------------------------------")

--------------------------------------------------------
Starting inference ...
--------------------------------------------------------


In [11]:
predict_examples = processor.get_test_examples(data_dir)
num_actual_predict_examples = len(predict_examples)

predict_file = os.path.join(output_dir, "predict.tf_record")

file_based_convert_examples_to_features(predict_examples, label_list,
                                        max_seq_length, tokenizer,
                                        predict_file)

tf.logging.info("***** Running prediction*****")
tf.logging.info("  Num examples = %d (%d actual, %d padding)",
                len(predict_examples), num_actual_predict_examples,
                len(predict_examples) - num_actual_predict_examples)
tf.logging.info("  Batch size = %d", predict_batch_size)

predict_drop_remainder = True if use_tpu else False
predict_input_fn = file_based_input_fn_builder(
    input_file=predict_file,
    seq_length=max_seq_length,
    is_training=False,
    drop_remainder=predict_drop_remainder)

result = estimator.predict(input_fn=predict_input_fn)

output_predict_file = os.path.join(output_dir, "test_results.tsv")

with tf.gfile.GFile(output_predict_file, "w") as writer:
    num_written_lines = 0
    tf.logging.info("***** Predict results *****")
    for (i, prediction) in enumerate(result):
        probabilities = prediction["probabilities"]
        if i >= num_actual_predict_examples:
            break
        output_line = "\t".join(
            str(class_probability)
            for class_probability in probabilities) + "\n"
        writer.write(output_line)
        num_written_lines += 1

INFO:tensorflow:Writing example 0 of 97320
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test-1
INFO:tensorflow:tokens: [CLS] jeff sessions is another one of trump ' s or ##well ##ian choices . he believes and has believed his entire career the exact opposite of what the position requires . [SEP]
INFO:tensorflow:input_ids: 101 5076 6521 2003 2178 2028 1997 8398 1005 1055 2030 4381 2937 9804 1012 2002 7164 1998 2038 3373 2010 2972 2476 1996 6635 4500 1997 2054 1996 2597 5942 1012 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow

INFO:tensorflow:  name = bert/encoder/layer_0/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_1/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/

INFO:tensorflow:  name = bert/encoder/layer_5/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/attention/output/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/intermediate/dense/kernel:0, shape = (768, 3072), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/intermediate/dense/bias:0, shape = (3072,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/dense/kernel:0, shape = (3072, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_5/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tenso

INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/query/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/key/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/self/value/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/dense/bias:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_10/attention/output/LayerNorm/beta:0, shape = (768,), *INIT_

In [12]:
end = time.time()
print("--------------------------------------------------------")
print("Inference complete in ", end - start, " seconds")
print("--------------------------------------------------------")

--------------------------------------------------------
Inference complete in  342.61383175849915  seconds
--------------------------------------------------------


In [13]:
sample_submission = pd.read_csv('./data/sample_submission.csv')
predictions = pd.read_csv('./test_results.tsv', header=None, sep='\t')

submission = pd.concat([sample_submission.iloc[:,0], predictions.iloc[:,1]], axis=1)
submission.columns = ['id','prediction']
submission.to_csv('submission.csv', index=False, header=True)